## 2.1.2 tf.data

  머신러닝에서 많은 시간을 소요하는 것은 데이터를 다루는 것이다. 일반적으로 데이터 분석, 전처리, 파이프라인 만드는 과정에서 약 70~80%의 시간을 소비한다 해도 과언이 아니다. 텐서플로우에서는 Dataset API를 활용하여, 단순한 연구자 뿐만 아니라 서비스화를 위해서 데이터에 대한 고민들을 해결 해 주고 있다. 학습 속도를 올리기 위해  GPU가 병목현상 없이 효율적으로 최적화를 지원하는 것 뿐만 아니라 이미지, 텍스트를 넘어 넘파이 (Numpy) 및 판다스(Pandas)데이터 구조가 활용하도록 지원이 가능하다.
  
  tf.data를 사용하기 위한 데이터를 활용하기 위해, IMDB 영화 리뷰 데이터셋을 불러오자. 데이터셋은 2만5천개의 리뷰의 데이터가 긍/부정으로 나누어져 있고, 전처리도 되어 있다. IMDB 영화 리뷰 데이터셋을 선정한 이유는 MNIST와 처럼 데이터셋을 tensorflow-keras를 활용하여 가장 손쉽게 불러오는 자연어 데이터이기 때문이다.

In [70]:
import os
import tensorflow as tf #텐서플로우 모듈 불러오기

from tensorflow.keras import preprocessing

import numpy as np

In [74]:
samples = ['너 오늘 이뻐 보인다', 
           '나는 오늘 기분이 더러워', 
           '끝내주는데, 좋은 일이 있나봐', 
           '나 좋은 일이 생겼어', 
           '아 오늘 진짜 짜증나', 
           '환상적인데, 정말 좋은거 같아']

label = [[1], [0], [1], [1],[0], [1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

sequences = preprocessing.sequence.pad_sequences(sequences, maxlen=max_len, padding='post')

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

print(word_index)
print(sequences)
print(label)

Found 20 unique tokens.
{'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}
[[ 4  1  5  6]
 [ 7  1  8  9]
 [10  2  3 11]
 [12  2  3 13]
 [14  1 15 16]
 [17 18 19 20]]
[[1], [0], [1], [1], [0], [1]]


In [85]:
MAX_LEN = 4
VOCAB_SIZE = len(word_index) #문장의 단어 사이즈
BATCH_SIZE = 2
NUM_EPOCHS = 1

def mapping_fn(X, Y=None):
    input, label = {'x': X}, Y
    return input, label

# Dataset에서 Iterator를 생성하여 초기화를 하거나 그 다음에 있는 tf.Tensor 객체를 생성하자.
#학습 셋을 Dataset에 입력한다 입력 형태는 numpy 형태이다.
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    dataset = dataset.batch(BATCH_SIZE) #배치구조로 데이터
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [81]:
# Dataset에서 Iterator를 생성하여 초기화를 하거나 그 다음에 있는 tf.Tensor 객체를 생성하자.
#학습 셋을 Dataset에 입력한다 입력 형태는 numpy 형태이다.
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
iterator = dataset.make_one_shot_iterator()
nextData = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(nextData))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


In [65]:
# 세션을 실행 시켜서 간단한 테스트를 진행 해 보자

with tf.Session() as sess:
    dataset = sess.run(train_input_fn())
    print("sentence index: {}".format(dataset[0]['x'])) # 입력 문장
    print("sentence Label: {}".format(dataset[1])) # 입력 문장 레이블 (긍/부정))

sentence index: [[4 1 5 6]
 [7 1 8 9]]
sentence Label: [[1]
 [0]]


In [87]:
with tf.Session() as sess:
    while True:
        try:
            dataset = sess.run(train_input_fn())
            print("sentence index: {} \n".format(dataset[0]['x'])) # 입력 문장
            print("sentence Label: {}".format(dataset[1])) # 입력 문장 레이블 (긍/부정))                   
        except tf.errors.OutOfRangeError:
            break

sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentence Label: [[1]
 [0]]
sentence index: [[4 1 5 6]
 [7 1 8 9]] 

sentenc

KeyboardInterrupt: 